---
## **Using Eigen face**
---

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn import datasets
import pandas as pd
from sklearn.cluster import DBSCAN, KMeans
import os
from sklearn import metrics
!pip install py7zr
import py7zr
import os
import cv2
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from torchvision import datasets
from tqdm import tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 922.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 8.3 MB/s eta 0:00:00


In [ ]:
!pip install kaggle

In [ ]:
os.environ['KAGGLE_USERNAME'] = ""
os.environ['KAGGLE_KEY'] = "198ad7f2"

In [ ]:
!kaggle datasets download -d hereisburak/pins-face-recognition

100% 372M/372M [00:19<00:00, 24.3MB/s]
100% 372M/372M [00:19<00:00, 20.3MB/s]


In [ ]:
import zipfile

def extract(path, folder):
  zip_ref = zipfile.ZipFile(path, 'r')
  zip_ref.extractall(folder)
  zip_ref.close()

base_folder = '/content/pins-face-recognition'
extract(base_folder + ".zip", base_folder)

for file in os.listdir(base_folder):
  name, ext = os.path.splitext(file)
  if ext == ".zip":
    extract(os.path.join(base_folder, file), base_folder)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install hdbscan
import hdbscan
def _make_cost_m(cm):
    s = np.max(cm)
    return (- cm + s)

from scipy.optimize import linear_sum_assignment as linear_assignment
from sklearn.metrics import confusion_matrix

In [ ]:
def calculate_f_measure(y,labels):
  cm = confusion_matrix(y, labels)
  indexes = linear_assignment(_make_cost_m(cm))
  # js = [e[1] for e in sorted(indexes, key=lambda x: x[0])]
  cm2 = cm[:, indexes[1]]

  acc = np.trace(cm2) / np.sum(cm2)
  precision = np.nanmean(np.diag(cm2) / np.sum(cm2, axis = 0))
  recall = np.nanmean(np.diag(cm2) / np.sum(cm2, axis = 1))
  f1 = 2 * precision * recall / (precision + recall)
  return f1

def evaluate_clustering(labels, predictions, beta=1.0):
  rand_index = metrics.rand_score(labels, predictions)
  nmi = metrics.normalized_mutual_info_score(labels, predictions)
  f_measure = calculate_f_measure(labels, predictions)
  print('Rand Index: %f, NMI: %f, F-measure: %f' % (rand_index, nmi, f_measure))
  return


In [ ]:
dataset = datasets.ImageFolder('/content/pins-face-recognition/105_classes_pins_dataset')

# Step 1: Data Collection and Preprocessing
def load_images(data):
    counter=1
    faces = []
    labels = []
    with tqdm(total=len(dataset)) as pbar:
      for image, label in data:
        image=np.array(image)
        image = cv2.resize(image, dsize=(160,160))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces.append( image)
        labels.append(label)
        counter+=1
        pbar.update(1)
    return faces, np.array(labels)

# Step 2: Compute the Mean Face
def compute_mean_face(faces):
    mean_face = np.mean(faces, axis=0)
    return mean_face

# Step 3: Compute the Covariance Matrix and Eigenfaces
def compute_eigenfaces(faces, num_components=100):
    # Flatten the 2D images into 1D arrays
    flattened_faces = np.array([face.flatten() for face in faces])

    # Standardize the data (optional but recommended)
    scaler = StandardScaler()
    standardized_faces = scaler.fit_transform(flattened_faces)

    # Perform PCA to compute eigenvalues and eigenvectors
    pca = PCA(n_components=num_components)
    eigenfaces = pca.fit_transform(standardized_faces)

    return eigenfaces, pca.components_, scaler


# Step 5: Clustering
def perform_clustering(eigenfaces, num_clusters):
    kmeans = KMeans(n_clusters=num_clusters)
    labels = kmeans.fit_predict(eigenfaces)
    return labels



data = dataset
num_clusters = 105  # Number of clusters for face clustering
num_components = 100  # Number of Eigenfaces to retain

# Step 1: Load Images
faces, labels = load_images(dataset)

# Step 2: Compute the Mean Face
mean_face = compute_mean_face(faces)

# Step 3: Compute Eigenfaces
eigenfaces, _, scaler = compute_eigenfaces(faces, num_components=num_components)


# Step 5: Clustering
cluster_labels = perform_clustering(eigenfaces, num_clusters)


100%|██████████| 17534/17534 [00:18<00:00, 967.42it/s] 
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
evaluate_clustering(labels, cluster_labels)

Rand Index: 0.979968, NMI: 0.149240, F-measure: 0.057479


In [ ]:
dbscan = DBSCAN(eps=0.5, min_samples=5, n_jobs=-1)


dbscan.fit(eigenfaces)

db_labels = dbscan.labels_

In [ ]:
evaluate_clustering(labels, db_labels)

Rand Index: 0.009861, NMI: 0.000000, F-measure: 0.011174


<ipython-input-13-a9a7f9a9b946>:8: RuntimeWarning: invalid value encountered in true_divide
  precision = np.nanmean(np.diag(cm2) / np.sum(cm2, axis = 0))
<ipython-input-13-a9a7f9a9b946>:9: RuntimeWarning: invalid value encountered in true_divide
  recall = np.nanmean(np.diag(cm2) / np.sum(cm2, axis = 1))


In [ ]:
!pip install hdbscan

In [ ]:
import hdbscan
hdbscan = hdbscan.HDBSCAN(min_cluster_size=3)
hdb_labels = hdbscan.fit_predict(eigenfaces)
evaluate_clustering(labels, hdb_labels)

Rand Index: 0.485122, NMI: 0.010245, F-measure: 0.027477


<ipython-input-13-a9a7f9a9b946>:8: RuntimeWarning: invalid value encountered in true_divide
  precision = np.nanmean(np.diag(cm2) / np.sum(cm2, axis = 0))
<ipython-input-13-a9a7f9a9b946>:9: RuntimeWarning: invalid value encountered in true_divide
  recall = np.nanmean(np.diag(cm2) / np.sum(cm2, axis = 1))
